In [1]:
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm_notebook as tqdm

In [2]:
pageinfo = pd.read_csv('/home3/usfb/build/output/page/1000-page-info.csv')
# politifact data with fake domain
postdata = pd.read_csv('/home3/usfb/analysis/analysis-fake-news/temp/post-match-domain/1000_page_politifact_domain.csv')

postdata_WithPageInfo = postdata.join(pageinfo[['page_id','category','type']].set_index('page_id'), 
                                                               on='page_id')

In [3]:
postdata_WithPageInfo["post_created_date_CT"] = pd.to_datetime(postdata_WithPageInfo["post_created_date_CT"])

In [4]:
postdata_WithPageInfo.columns

Index(['page_id', 'page_name', 'post_id', 'post_type', 'post_name',
       'post_message', 'post_caption', 'post_picture', 'post_link',
       'post_description', 'post_reactions', 'post_likes', 'post_comments',
       'post_shares', 'post_created_time_CT', 'post_updated_time_CT',
       'post_created_date_CT', 'post_updated_date_CT', 'post_created_time',
       'post_updated_time', 'page_talking_about_count', 'domain', 'category',
       'type'],
      dtype='object')

In [5]:
def filter_post_data( start_date, end_date, date_col ="post_created_date_CT"):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    return postdata_WithPageInfo[(postdata_WithPageInfo[date_col] >= start_date) & (postdata_WithPageInfo[date_col] <= end_date)]

In [6]:
from nltk.corpus import wordnet
import nltk

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [7]:
import string
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm import tqdm_notebook as tqdm
import re

def tokenize_with_lemmentize(document_list ,lemmentize = True):

    
    tokenized_post = []
    lemmatizer = WordNetLemmatizer()
    
    removed = stopwords.words('english') + list(string.punctuation)
    for i in tqdm(document_list):
        post = re.sub(r'http\S+', '', str(i))
        tokens = word_tokenize(post)
        words = [word.lower() for word in tokens if word.isalpha() and word not in removed]
        if lemmentize:
            words_lemmantized = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in words]
            tokenized_post.append(words_lemmantized)
        else:
            tokenized_post.append(words)
    return tokenized_post

In [8]:
time_vec = ["2016/03", "2016/04", "2016/05", "2016/06", "2016/07", "2016/08", "2016/09"]

In [9]:
from gensim.models.word2vec import Word2Vec
trump_dict = {}
hillary_dict = {}
obama_dict = {}
black_dict = {}
immigrant_dict = {}
nuclear_dict = {}
isis_dict = {}
mexico_dict = {}
muslim_dict = {}
president_dict = {}
police_dict = {}

for i in range(len(time_vec)-1):
    monthly_data = filter_post_data(time_vec[i], time_vec[i+1])
    tokenized_monthly_data = tokenize_with_lemmentize(monthly_data["post_message"])
    model_d300 = Word2Vec(tokenized_monthly_data, size=300, iter=15)
    trump_dict[time_vec[i]] = model_d300.most_similar("trump")
    hillary_dict[time_vec[i]] = model_d300.most_similar("hillary")
    obama_dict[time_vec[i]] = model_d300.most_similar("obama")
    black_dict[time_vec[i]] = model_d300.most_similar("black")
    immigrant_dict[time_vec[i]] = model_d300.most_similar("immigrant")
    nuclear_dict[time_vec[i]] = model_d300.most_similar("nuclear")
    mexico_dict[time_vec[i]] = model_d300.most_similar("mexico")
    isis_dict[time_vec[i]] = model_d300.most_similar("isi")
    muslim_dict[time_vec[i]] = model_d300.most_similar("muslim")
    president_dict[time_vec[i]] = model_d300.most_similar("president")
    police_dict[time_vec[i]] = model_d300.most_similar("police")    
    print("done", time_vec[i])
    
    #mulsim, police, president
    

/home3/b02303004/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
/home3/b02303004/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
/home3/b02303004/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
/home3/b02303004/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
/home3/b02303004/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use sel

done 2016/03



done 2016/04



done 2016/05



done 2016/06



done 2016/07



done 2016/08


In [10]:
pd.DataFrame(trump_dict)

,2016/03,2016/04,2016/05,2016/06,2016/07,2016/08
0,"(hometown, 0.8550615906715393)","(gop, 0.8744391202926636)","(donald, 0.9370287656784058)","(donald, 0.9668252468109131)","(donald, 0.9118469953536987)","(donald, 0.9207526445388794)"
1,"(republican, 0.851472795009613)","(supporter, 0.8493265509605408)","(supporter, 0.8628761172294617)","(supporter, 0.9500084519386292)","(win, 0.8329495191574097)","(president, 0.799026608467102)"
2,"(dethrone, 0.8431159853935242)","(nomination, 0.8454573154449463)","(clinton, 0.8603715896606445)","(nominee, 0.9347062110900879)","(hat, 0.7803099751472473)","(election, 0.7948530912399292)"
3,"(gop, 0.8371431231498718)","(bernie, 0.8406292200088501)","(appeal, 0.8570966124534607)","(president, 0.9121201038360596)","(speech, 0.7692674994468689)","(win, 0.7767789959907532)"
4,"(endorse, 0.8246254920959473)","(stunt, 0.8352288007736206)","(hillary, 0.8366497755050659)","(about, 0.8882293701171875)","(square, 0.7679250836372375)","(presidential, 0.7729327082633972)"
5,"(ted, 0.8184956312179565)","(republican, 0.8340886235237122)","(conservatism, 0.8315783739089966)","(we, 0.8814998865127563)","(supporter, 0.7498733997344971)","(hopeful, 0.7715611457824707)"
6,"(threat, 0.8160818815231323)","(rally, 0.8305776119232178)","(playin, 0.8289270401000977)","(victory, 0.880535364151001)","(unlikely, 0.7496288418769836)","(democrat, 0.7489191293716431)"
7,"(protest, 0.8160649538040161)","(ted, 0.8201252222061157)","(election, 0.8142187595367432)","(rally, 0.8797429800033569)","(vote, 0.7377030253410339)","(landslide, 0.7480372190475464)"
8,"(swing, 0.8061093688011169)","(donald, 0.8132543563842773)","(vote, 0.812896728515625)","(hate, 0.8759112358093262)","(presidency, 0.7293853759765625)","(hillary, 0.7424107789993286)"
9,"(rally, 0.8039082288742065)","(lover, 0.798482358455658)","(wonder, 0.809624969959259)","(obama, 0.8758997917175293)","(bernie, 0.7208196520805359)","(candidate, 0.7414817214012146)"


In [11]:
pd.DataFrame(hillary_dict)

,2016/03,2016/04,2016/05,2016/06,2016/07,2016/08
0,"(clinton, 0.8207790851593018)","(clinton, 0.929862916469574)","(supporter, 0.9639663696289062)","(clinton, 0.964397132396698)","(bill, 0.9455478191375732)","(foundation, 0.9219242334365845)"
1,"(bill, 0.8169132471084595)","(associate, 0.8514461517333984)","(bernie, 0.9613553881645203)","(bernie, 0.9338071346282959)","(clinton, 0.9425305128097534)","(clinton, 0.9105269908905029)"
2,"(lie, 0.8051995038986206)","(bill, 0.832710862159729)","(hildebeest, 0.9362407326698303)","(we, 0.9281870126724243)","(prison, 0.9207329750061035)","(bill, 0.8793078660964966)"
3,"(closing, 0.7865649461746216)","(drop, 0.8287142515182495)","(appeal, 0.9247264266014099)","(how, 0.9242109060287476)","(comey, 0.9123255014419556)","(campaign, 0.8722040057182312)"
4,"(increase, 0.7812023758888245)","(author, 0.7917100191116333)","(clinton, 0.9219204187393188)","(wow, 0.9120492935180664)","(benghazi, 0.9058942794799805)","(health, 0.8691850900650024)"
5,"(scandalous, 0.7798377871513367)","(benghazi, 0.788425087928772)","(vote, 0.9157329797744751)","(epic, 0.9074262380599976)","(director, 0.8957643508911133)","(email, 0.8637784719467163)"
6,"(unbelievably, 0.7664677500724792)","(fbi, 0.7857806086540222)","(fwd, 0.9085311889648438)","(outdid, 0.9058667421340942)","(campaign, 0.8940743207931519)","(election, 0.8521318435668945)"
7,"(benghazi, 0.7608048915863037)","(email, 0.779797375202179)","(holy, 0.907490074634552)","(wo, 0.899250864982605)","(email, 0.8744220733642578)","(scandal, 0.8505887985229492)"
8,"(backstage, 0.7488067150115967)","(ouch, 0.7739375233650208)","(skin, 0.9051986932754517)","(hildebeest, 0.8991643190383911)","(charge, 0.8703196048736572)","(soros, 0.8489767909049988)"
9,"(lately, 0.7441550493240356)","(democrat, 0.7625199556350708)","(here, 0.9038321375846863)","(blast, 0.8989136219024658)","(loretta, 0.8637451529502869)","(democratic, 0.8463799357414246)"


In [12]:
pd.DataFrame(obama_dict)

,2016/03,2016/04,2016/05,2016/06,2016/07,2016/08
0,"(president, 0.8923858404159546)","(barack, 0.8731546401977539)","(barack, 0.9367908239364624)","(barack, 0.9727113246917725)","(president, 0.8670456409454346)","(isi, 0.8784763813018799)"
1,"(lady, 0.8805394172668457)","(michelle, 0.8706600069999695)","(president, 0.9288972616195679)","(president, 0.9626911282539368)","(barack, 0.8432602882385254)","(u, 0.8659222722053528)"
2,"(first, 0.86775803565979)","(treason, 0.8619562983512878)","(power, 0.9134594202041626)","(allow, 0.9570749402046204)","(isi, 0.7852820754051208)","(for, 0.8638995885848999)"
3,"(country, 0.8511186838150024)","(administration, 0.8456803560256958)","(for, 0.9041968584060669)","(to, 0.9293708801269531)","(united, 0.776667594909668)","(who, 0.8622912168502808)"
4,"(compare, 0.8490743637084961)","(president, 0.8370089530944824)","(law, 0.9003915786743164)","(military, 0.9292024970054626)","(hell, 0.773594081401825)","(to, 0.8511129021644592)"
5,"(general, 0.8440219163894653)","(election, 0.8322949409484863)","(with, 0.8996103405952454)","(flag, 0.9246649146080017)","(administration, 0.7685525417327881)","(barack, 0.8448841571807861)"
6,"(nancy, 0.8410084247589111)","(economy, 0.8012745380401611)","(about, 0.8995693922042847)","(war, 0.9135622978210449)","(still, 0.7625365853309631)","(about, 0.8180648684501648)"
7,"(barack, 0.8383817672729492)","(suspend, 0.8004518747329712)","(martial, 0.8976858258247375)","(radical, 0.9069441556930542)","(military, 0.7603616714477539)","(military, 0.8139539957046509)"
8,"(reagan, 0.8363125324249268)","(appointee, 0.7959905862808228)","(dollar, 0.8787299394607544)","(islam, 0.9050294756889343)","(review, 0.7564769983291626)","(troop, 0.8093885183334351)"
9,"(michelle, 0.8245453238487244)","(defect, 0.7935705184936523)","(government, 0.8786781430244446)","(arm, 0.9043657779693604)","(institute, 0.752836287021637)","(american, 0.8056337833404541)"


In [13]:
pd.DataFrame(immigrant_dict)

,2016/03,2016/04,2016/05,2016/06,2016/07,2016/08
0,"(worried, 0.9762831926345825)","(slaughter, 0.9813516736030579)","(perpetuate, 0.9919471740722656)","(pharmacy, 0.9880784749984741)","(illegal, 0.9390460848808289)","(extremely, 0.9875128269195557)"
1,"(tie, 0.9720911383628845)","(cruise, 0.9742547273635864)","(faith, 0.9918906688690186)","(emergency, 0.9848446249961853)","(violates, 0.9238535165786743)","(promote, 0.9860358238220215)"
2,"(devout, 0.9661487936973572)","(kidnap, 0.970796525478363)","(operation, 0.9916554093360901)","(text, 0.9845790863037109)","(aclu, 0.9045213460922241)","(havoc, 0.9850438833236694)"
3,"(housing, 0.9621595740318298)","(russian, 0.9689689874649048)","(feeling, 0.9905747771263123)","(rumor, 0.9828404188156128)","(backing, 0.8998275995254517)","(boycott, 0.9846732020378113)"
4,"(charity, 0.9617413282394409)","(hundred, 0.9661952257156372)","(mission, 0.9901725649833679)","(concern, 0.9826854467391968)","(jeanine, 0.8948674201965332)","(tension, 0.9839441776275635)"
5,"(prayer, 0.9577265977859497)","(gay, 0.9648935794830322)","(delivers, 0.9896056652069092)","(danger, 0.9824475049972534)","(bluelivesmatter, 0.8885104656219482)","(racial, 0.9829906821250916)"
6,"(teresa, 0.9553418755531311)","(slave, 0.9639353156089783)","(rise, 0.9893810749053955)","(patron, 0.9824331402778625)","(possible, 0.8845081329345703)","(russian, 0.9826244115829468)"
7,"(stab, 0.9523270726203918)","(egypt, 0.9631237387657166)","(storm, 0.9887707233428955)","(philadelphia, 0.9823799729347229)","(pirro, 0.8831260204315186)","(film, 0.982326328754425)"
8,"(betrayed, 0.9506781101226807)","(crawl, 0.9624338746070862)","(morris, 0.9886534810066223)","(surround, 0.9819736480712891)","(practice, 0.8810672163963318)","(photograph, 0.9821658134460449)"
9,"(execute, 0.9493715167045593)","(core, 0.9596349000930786)","(golf, 0.988578200340271)","(delighted, 0.981423556804657)","(course, 0.8807876110076904)","(join, 0.981867790222168)"


In [14]:
pd.DataFrame(nuclear_dict)

,2016/03,2016/04,2016/05,2016/06,2016/07,2016/08
0,"(prepare, 0.9807527661323547)","(combat, 0.9639673233032227)","(economic, 0.9896737337112427)","(those, 0.9927159547805786)","(style, 0.9837737679481506)","(against, 0.9935657978057861)"
1,"(civil, 0.9686626195907593)","(drone, 0.9610919952392578)","(currency, 0.9893409609794617)","(conspiracy, 0.9888414144515991)","(sea, 0.9836180210113525)","(professor, 0.9930511116981506)"
2,"(february, 0.9620363712310791)","(them, 0.9588984251022339)","(most, 0.9871427416801453)","(illuminati, 0.9869897961616516)","(creature, 0.9776164293289185)","(fake, 0.9926608800888062)"
3,"(flag, 0.9599394202232361)","(mayor, 0.9588556885719299)","(market, 0.9843053817749023)","(increase, 0.9863677024841309)","(pilot, 0.9770082235336304)","(audio, 0.9908148646354675)"
4,"(admiral, 0.9522742033004761)","(strip, 0.958454966545105)","(funding, 0.9842830300331116)","(underground, 0.9862411022186279)","(destabilize, 0.9756361842155457)","(global, 0.9907280802726746)"
5,"(against, 0.9512999057769775)","(choke, 0.9568692445755005)","(putin, 0.9839150905609131)","(base, 0.985408365726471)","(nuke, 0.9752031564712524)","(technology, 0.9902135133743286)"
6,"(nuke, 0.9464329481124878)","(bee, 0.9559858441352844)","(top, 0.9838367104530334)","(declares, 0.982552170753479)","(globalist, 0.9744101762771606)","(herself, 0.9901790618896484)"
7,"(mass, 0.9446150660514832)","(threatens, 0.9544357657432556)","(our, 0.9812600612640381)","(ops, 0.9825344681739807)","(moon, 0.9739931225776672)","(nuke, 0.9891557693481445)"
8,"(dc, 0.9409556984901428)","(killer, 0.9542752504348755)","(peter, 0.9812195301055908)","(criminal, 0.9809274077415466)","(earth, 0.9738835692405701)","(allegedly, 0.9884905815124512)"
9,"(prisoner, 0.9406411647796631)","(decides, 0.9523902535438538)","(global, 0.9812103509902954)","(disclosure, 0.9807463884353638)","(music, 0.9723381400108337)","(burning, 0.9882477521896362)"


In [15]:
pd.DataFrame(mexico_dict)

,2016/03,2016/04,2016/05,2016/06,2016/07,2016/08
0,"(cloud, 0.9810633659362793)","(colorado, 0.9294183254241943)","(saudi, 0.9938427209854126)","(bloody, 0.9867510795593262)","(myth, 0.9868048429489136)","(fisherman, 0.9893580675125122)"
1,"(tragically, 0.9777535200119019)","(jersey, 0.9293662905693054)","(november, 0.9930213689804077)","(patron, 0.9837044477462769)","(stab, 0.9861763715744019)","(daylight, 0.9875056743621826)"
2,"(cashless, 0.9763671159744263)","(lover, 0.919571042060852)","(rag, 0.9928001761436462)","(seeker, 0.9836426973342896)","(meanwhile, 0.9849104881286621)","(broad, 0.9859050512313843)"
3,"(pinpoint, 0.9759034514427185)","(sole, 0.9167255163192749)","(build, 0.9918442368507385)","(alarm, 0.9835867881774902)","(inmate, 0.9848743081092834)","(rope, 0.9853055477142334)"
4,"(through, 0.9745192527770996)","(dream, 0.9116816520690918)","(politician, 0.9916155934333801)","(pharmacy, 0.9833067655563354)","(bleeding, 0.9822390079498291)","(panic, 0.9852014183998108)"
5,"(energy, 0.9743884205818176)","(name, 0.9086628556251526)","(doubt, 0.9914331436157227)","(prayer, 0.9832684397697449)","(savagely, 0.9813436269760132)","(manage, 0.9848544597625732)"
6,"(gold, 0.9742025136947632)","(contender, 0.9077662825584412)","(director, 0.9906115531921387)","(regular, 0.9829539060592651)","(defy, 0.980338454246521)","(tradition, 0.9848089218139648)"
7,"(sky, 0.9741840362548828)","(victory, 0.9066019058227539)","(alive, 0.9905475974082947)","(emergency, 0.9827756285667419)","(building, 0.9781259894371033)","(somali, 0.9844512343406677)"
8,"(replace, 0.9737730026245117)","(purpose, 0.9065898656845093)","(warm, 0.9902858734130859)","(twin, 0.9818699955940247)","(ranger, 0.9780207872390747)","(surface, 0.9843462109565735)"
9,"(apple, 0.9730232954025269)","(ballot, 0.9051639437675476)","(captain, 0.9902803897857666)","(threw, 0.9810179471969604)","(angeles, 0.9776176810264587)","(circumstance, 0.9841834306716919)"


In [16]:
pd.DataFrame(isis_dict)

,2016/03,2016/04,2016/05,2016/06,2016/07,2016/08
0,"(sympathizer, 0.8637944459915161)","(against, 0.9192862510681152)","(tell, 0.9581538438796997)","(radical, 0.9716985821723938)","(ban, 0.9156994223594666)","(federal, 0.9553745985031128)"
1,"(implicate, 0.8601115942001343)","(confederate, 0.9141080379486084)","(charge, 0.9554691314697266)","(terror, 0.9464678168296814)","(sheriff, 0.91375732421875)","(control, 0.9541118741035461)"
2,"(world, 0.8590621948242188)","(with, 0.9074402451515198)","(scum, 0.9443148970603943)","(his, 0.9439318180084229)","(admiral, 0.9057021141052246)","(troop, 0.9459179639816284)"
3,"(prepare, 0.858099102973938)","(saudi, 0.891238808631897)","(blm, 0.9397938251495361)","(wave, 0.94355309009552)","(blast, 0.9043293595314026)","(treason, 0.9409822821617126)"
4,"(american, 0.8534314036369324)","(war, 0.8900763988494873)","(criminal, 0.9362395405769348)","(audacity, 0.9427881240844727)","(declare, 0.8985435366630554)","(gun, 0.9371228814125061)"
5,"(nuclear, 0.8521993160247803)","(government, 0.886246919631958)","(racist, 0.9362373352050781)","(flag, 0.9343634843826294)","(s, 0.8981940746307373)","(sniper, 0.9287830591201782)"
6,"(include, 0.8472573757171631)","(over, 0.8833121061325073)","(blame, 0.9328732490539551)","(threat, 0.9221935272216797)","(islam, 0.8973211646080017)","(who, 0.9285337924957275)"
7,"(flag, 0.8414421081542969)","(ag, 0.8829882740974426)","(jeanine, 0.9323025345802307)","(jihad, 0.9191581606864929)","(clarke, 0.893378734588623)","(prison, 0.9262977242469788)"
8,"(law, 0.8368632793426514)","(million, 0.8818281888961792)","(hate, 0.9281222820281982)","(terrorist, 0.9179085493087769)","(rhetoric, 0.8908547759056091)","(king, 0.9250178933143616)"
9,"(top, 0.8303869962692261)","(terrorist, 0.8817061185836792)","(then, 0.9266980886459351)","(into, 0.9168624877929688)","(hell, 0.8888813257217407)","(executive, 0.9150059223175049)"


In [17]:
pd.DataFrame(muslim_dict)

,2016/03,2016/04,2016/05,2016/06,2016/07,2016/08
0,"(asylum, 0.900188684463501)","(migrant, 0.9287776350975037)","(migrant, 0.9148340821266174)","(new, 0.8946600556373596)","(migrant, 0.9226791858673096)","(woman, 0.9086319208145142)"
1,"(seeker, 0.8977164030075073)","(an, 0.8303179740905762)","(asylum, 0.907802164554596)","(turf, 0.8752000331878662)","(mother, 0.8850940465927124)","(migrant, 0.879019558429718)"
2,"(migrant, 0.883112907409668)","(force, 0.8208457827568054)","(seeker, 0.8966419100761414)","(migrant, 0.871248722076416)","(young, 0.8822963237762451)","(a, 0.8585602045059204)"
3,"(group, 0.8831078410148621)","(erect, 0.8199111819267273)","(recently, 0.8770729899406433)","(group, 0.8647496700286865)","(spot, 0.8795883655548096)","(refugee, 0.855136513710022)"
4,"(refugee, 0.8612107634544373)","(gang, 0.8195391893386841)","(somali, 0.8542468547821045)","(community, 0.8553208112716675)","(an, 0.878449559211731)","(rap, 0.8542351722717285)"
5,"(gang, 0.8549729585647583)","(tent, 0.8055516481399536)","(pervert, 0.8500789403915405)","(brutally, 0.8488115668296814)","(teen, 0.877609372138977)","(burka, 0.848091721534729)"
6,"(islamophobic, 0.8371802568435669)","(islamic, 0.800325870513916)","(florida, 0.8465421795845032)","(york, 0.8486314415931702)","(woman, 0.8771331906318665)","(after, 0.8200326561927795)"
7,"(rap, 0.834139883518219)","(depend, 0.7948905229568481)","(married, 0.8436819314956665)","(become, 0.8472265601158142)","(increasingly, 0.8758063316345215)","(married, 0.8188444375991821)"
8,"(father, 0.8191035389900208)","(brutally, 0.7939667701721191)","(california, 0.8411681056022644)","(force, 0.8351431488990784)","(sexually, 0.8661725521087646)","(men, 0.8101990222930908)"
9,"(brutally, 0.8116226196289062)","(stab, 0.7888289093971252)","(an, 0.8390143513679504)","(mob, 0.8331944346427917)","(gang, 0.8569425344467163)","(recently, 0.807709813117981)"


In [18]:
pd.DataFrame(black_dict)

,2016/03,2016/04,2016/05,2016/06,2016/07,2016/08
0,"(life, 0.8740509152412415)","(matter, 0.8964183330535889)","(matter, 0.9729547500610352)","(white, 0.8995488882064819)","(matter, 0.96494460105896)","(matter, 0.9228376746177673)"
1,"(matter, 0.8323378562927246)","(crime, 0.8938106298446655)","(thug, 0.9723945260047913)","(murder, 0.8862236142158508)","(life, 0.9479628801345825)","(life, 0.914386510848999)"
2,"(movement, 0.7326319217681885)","(activist, 0.890060305595398)","(life, 0.9492450952529907)","(into, 0.8716334700584412)","(chooses, 0.9443879723548889)","(riot, 0.9084621667861938)"
3,"(allege, 0.6984031796455383)","(thug, 0.8883088827133179)","(panther, 0.9479621648788452)","(jihad, 0.8654398918151855)","(panther, 0.9378587007522583)","(protest, 0.900939404964447)"
4,"(allow, 0.6904773712158203)","(panther, 0.8874728679656982)","(white, 0.9313727021217346)","(kill, 0.8642213940620422)","(represent, 0.9310489892959595)","(panther, 0.8912547826766968)"
5,"(thug, 0.6862193942070007)","(life, 0.8786330819129944)","(try, 0.9243769645690918)","(audacity, 0.8616337180137634)","(thug, 0.9249669909477234)","(sentiment, 0.8858497142791748)"
6,"(attack, 0.6653295755386353)","(violent, 0.8651943802833557)","(racist, 0.9008010625839233)","(shooter, 0.8610203266143799)","(lose, 0.9122868180274963)","(blm, 0.8764420747756958)"
7,"(fade, 0.6588146686553955)","(hateful, 0.8570349216461182)","(mob, 0.899716854095459)","(jihadist, 0.8576474189758301)","(community, 0.9082757234573364)","(movement, 0.8755433559417725)"
8,"(racist, 0.6514230370521545)","(community, 0.8528236150741577)","(blm, 0.8990515470504761)","(nightclub, 0.8551925420761108)","(repeatedly, 0.9078356027603149)","(baton, 0.8735110759735107)"
9,"(suspect, 0.6492112874984741)","(additionally, 0.8357424736022949)","(kill, 0.8896377682685852)","(power, 0.8547123074531555)","(minority, 0.8905657529830933)","(clown, 0.8686947822570801)"


In [19]:
pd.DataFrame(president_dict)

,2016/03,2016/04,2016/05,2016/06,2016/07,2016/08
0,"(barack, 0.9477647542953491)","(barack, 0.9642857313156128)","(barack, 0.9565902948379517)","(barack, 0.9756418466567993)","(barack, 0.9542608857154846)","(barack, 0.9545921087265015)"
1,"(first, 0.896009087562561)","(may, 0.9008707404136658)","(obama, 0.9288973808288574)","(obama, 0.962691068649292)","(united, 0.8948626518249512)","(hopeful, 0.866559624671936)"
2,"(obama, 0.8923857808113098)","(suspend, 0.8430790901184082)","(reveals, 0.8873435854911804)","(allow, 0.9488692283630371)","(obama, 0.8670456409454346)","(silence, 0.8601585626602173)"
3,"(compare, 0.8784258365631104)","(obama, 0.8370089530944824)","(u, 0.8710343837738037)","(supporter, 0.9420042037963867)","(state, 0.8637690544128418)","(secretary, 0.856378436088562)"
4,"(military, 0.8573760390281677)","(presidential, 0.8249564170837402)","(knowingly, 0.8694887161254883)","(blame, 0.9343178868293762)","(suspend, 0.8576100468635559)","(democrat, 0.8384430408477783)"
5,"(united, 0.854693591594696)","(defy, 0.7966004610061646)","(remove, 0.8660829067230225)","(no, 0.931533932685852)","(secretary, 0.8540838956832886)","(show, 0.8371164798736572)"
6,"(plan, 0.8497663140296936)","(appointee, 0.7940026521682739)","(appointee, 0.865648627281189)","(war, 0.9287348985671997)","(election, 0.8423647880554199)","(accountable, 0.8358001708984375)"
7,"(castro, 0.844322681427002)","(michelle, 0.7839260101318359)","(administration, 0.865436315536499)","(they, 0.9283779859542847)","(administration, 0.8292120695114136)","(government, 0.8312268257141113)"
8,"(michelle, 0.8333046436309814)","(knowingly, 0.7529957890510559)","(leader, 0.8570316433906555)","(your, 0.9275220036506653)","(campaign, 0.8207030296325684)","(presidency, 0.8241604566574097)"
9,"(weekly, 0.8228369951248169)","(idiotic, 0.748265266418457)","(martial, 0.8556772470474243)","(america, 0.9269942045211792)","(accountable, 0.8200792074203491)","(putin, 0.8238953351974487)"
